In [1]:
import pandas as pd
import numpy as np
import torch
import os
from utils import data

dataset_path = './dataset/'
dataset_files = ['TcData.csv' ,'BsData.csv']

pd.set_option("display.max_columns", None)

dfs = dict([])
for file in dataset_files:
    df = pd.read_csv(os.path.join(dataset_path,file), na_values=0)
    df = df.rename(columns={'Composition': 'formula'})
    df.name = file[:2]
    dfs[df.name] = df

load_df = True
for df in dfs.values():
    df = data.featurize_data(df,load_existing_df=True, pkl_path = os.path.join(dataset_path, f"./df_data_{df.name}.pkl"))
    
from utils.surrogate import surrogate_model

Tc_surrogate = surrogate_model(name='Tc', df=dfs['Tc'])
Bs_surrogate = surrogate_model(name='Bs', df=dfs['Bs'])

Tc_surrogate.cleanup_df(drop_NaN = False, 
                        drop_col_with_NaN = True)
Bs_surrogate.cleanup_df(drop_NaN = False, 
                        drop_col_with_NaN = True)

surrogates = [Tc_surrogate, Bs_surrogate] 

In [2]:
data.describe_non_zero_mean(Tc_surrogate.df)

,Thickness (mu m),Tc (K)
count,220.000000,220.000000
mean,21.677273,599.232518
std,3.351607,73.767177
min,17.500000,370.000000
25%,20.000000,587.000000
50%,20.000000,605.983795
75%,22.500000,626.768258
max,30.000000,769.000000


In [3]:
data.describe_non_zero_mean(Bs_surrogate.df)

,Thickness (mu m),Bs (T)
count,294.000000,294.000000
mean,23.423469,1.556457
std,5.147990,0.281092
min,13.000000,0.259860
25%,20.000000,1.473255
50%,22.000000,1.650000
75%,25.000000,1.738235
max,76.000000,1.888650


In [4]:
data.describe_non_zero_mean(pd.concat([Tc_surrogate.df, Bs_surrogate.df]))

,Thickness (mu m),Tc (K),Bs (T)
count,514.000000,220.000000,294.000000
mean,22.676070,599.232518,1.556457
std,4.547522,73.767177,0.281092
min,13.000000,370.000000,0.259860
25%,20.000000,587.000000,1.473255
50%,21.000000,605.983795,1.650000
75%,25.000000,626.768258,1.738235
max,76.000000,769.000000,1.888650


In [51]:
df = surrogates[0].df

subset = ['Nb', 'Cu', 'Si', 'B', 'Fe']
df_subset = data.subset_df_by_elements(pd.concat([Tc_surrogate.df, Bs_surrogate.df]), subset)
data.describe_non_zero_mean(df_subset)

,Thickness (mu m),Tc (K),B,Si,V,Fe,Ni,Cu,Nb,Mo,Bs (T)
count,132.000000,78.000000,132.000000,132.000000,2.000000,132.000000,3.0000,132.000000,132.000000,2.000000,54.000000
mean,21.954545,619.232528,0.091839,0.120650,0.010000,0.749189,0.0133,0.009671,0.028045,0.010000,1.378643
std,2.648242,34.815406,0.017761,0.037582,0.007071,0.024254,0.0000,0.001643,0.006349,0.007071,0.277504
min,20.000000,541.000000,0.070000,0.040000,0.005000,0.720000,0.0133,0.002100,0.004000,0.005000,0.680000
25%,20.000000,600.557960,0.075000,0.091600,0.007500,0.735000,0.0133,0.010000,0.029412,0.007500,1.182972
50%,21.000000,618.472175,0.090000,0.135000,0.010000,0.735000,0.0133,0.010000,0.030000,0.010000,1.308785
75%,22.500000,625.880637,0.092409,0.150000,0.012500,0.759000,0.0133,0.010000,0.030000,0.012500,1.618920
max,30.000000,769.000000,0.130000,0.155000,0.015000,0.803922,0.0133,0.013000,0.040000,0.015000,1.801110


In [5]:
def EF_presence(df):
    stat = df.astype(bool).sum(axis=0)
    return stat[stat>0][4:]

EF_presence(surrogates[0].df)

Series([], dtype: int64)

In [8]:
EF_presence(surrogates[1].df)

B     279
C     110
Al     20
Si    226
P     168
V       2
Cr      2
Fe    294
Co     20
Ni      5
Cu    178
Ga      8
Ge      7
Zr     27
Nb     67
Mo      4
Sn      3
Hf     17
dtype: int64